# 1. Introduction
# ICU survival modeling, Part 2: Cox and predictive models

**Approach**
- Teach and compare three models using the same preprocessing and the same train, validation, and test sets  
  - Cox proportional hazards for time-to-event  
  - Decision Tree classifier at fixed horizons  
  - Random Forest classifier at fixed horizons

**Learning goals**
- Build fair, fixed-horizon predictions at 7, 30, and 60 days from ICU admission  
- Understand discrimination vs calibration and why both matter clinically  
- See interpretability trade-offs across Cox, a single tree, and an ensemble forest  
- Practice leakage-free preprocessing with scikit-learn Pipelines

**Data**
- PhysioNet CinC Challenge 2012 ICU cohort, set A, 4000 stays, first 48 h features plus outcomes  
- Outcomes available  
  - Length of stay in days  
  - Survival in days up to 2 years  
  - In-hospital death indicator

**Fair comparison plan**
1) One reproducible split into train, validation, and test used by all models  
2) Identical preprocessing via a single scikit-learn ColumnTransformer  
3) Fixed-horizon evaluation at 7, 30, 60 days on the same evaluable patients per horizon  
4) Report AUROC, area under precision-recall, Brier score, and compact calibration by bins

**Clinical reading of metrics**
- Discrimination ranks who is higher risk  
- Calibration asks if predicted risk matches observed risk at a threshold relevant for action

References  
- Official challenge description and variable definitions, including Survival and In-hospital death  


# 2. Setup


In [13]:
## 2) Code cell — Setup and reproducibility

# 2. Setup and reproducibility

# Standard library
from pathlib import Path
import warnings
import math
import os
import sys
import random

# Third-party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

# Modeling
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.calibration import calibration_curve

# Lifelines
from lifelines import CoxPHFitter

# Our helpers
sys.path.append("/mnt/data")
import utils  # uses build_preprocessor, detect_feature_types, to_dataframe, labeling and calibration helpers

# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Pandas and plotting defaults
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 120)
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (7, 4)

# Paths
DATA_PATH = Path("/mnt/data/PhysionetChallenge2012-set-a.csv.gz")

print("Versions")
print("numpy", np.__version__)
print("pandas", pd.__version__)
import sklearn
print("scikit-learn", sklearn.__version__)
import lifelines
print("lifelines", lifelines.__version__)


Versions
numpy 2.3.3
pandas 2.3.3
scikit-learn 1.7.2
lifelines 0.30.0


# 3. Load data and define outcomes

In [5]:
# 3. Load data and define variables  [in-hospital death endpoint]

# --- 1. Load Data ---
PATH = "PhysionetChallenge2012-set-a.csv.gz"

# Simple check to ensure the data file exists before trying to load it
if not os.path.exists(PATH):
    raise FileNotFoundError(
        f"Error: The data file was not found at '{PATH}'. "
        "Please ensure the dataset is in the correct directory."
    )

raw = pd.read_csv(PATH, compression="gzip")
    
# Basic sanity checks
required_cols = {"Length_of_stay", "Survival", "In-hospital_death"}
missing = required_cols.difference(raw.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# Endpoint locked to in-hospital death
# Duration is time from ICU admission to discharge or in-hospital death
# Event is 1 if died in hospital, 0 if discharged alive
def make_outcomes_in_hospital(df: pd.DataFrame) -> pd.DataFrame:
    y = pd.DataFrame(index=df.index)
    los = pd.to_numeric(df["Length_of_stay"], errors="coerce").astype(float)
    los = np.clip(los, 0.0, None)  # clip negatives to 0 days
    event = pd.to_numeric(df["In-hospital_death"], errors="coerce").fillna(0).astype(int)
    y["duration_days"] = los
    y["event_death"] = event
    return y

y = make_outcomes_in_hospital(raw)

# Feature frame: drop outcome columns and obvious identifiers to avoid leakage
drop_cols = ["In-hospital_death", "Survival", "Length_of_stay", "recordid"]
X = raw.drop(columns=[c for c in drop_cols if c in raw.columns], errors="ignore").copy()

print("Outcome head")
display(y.head(3).style)
print("Features shape", X.shape)

# Quick outcome summary
event_rate = float(y["event_death"].mean())
duration = y["duration_days"].to_numpy()
iqr = float(np.percentile(duration, 75) - np.percentile(duration, 25))
print(f"Event rate: {event_rate:.3f}")
print(f"Follow-up days: median {float(np.median(duration)):.1f}  IQR {iqr:.1f}")


Outcome head


,duration_days,event_death
0,5.000000,0
1,8.000000,0
2,19.000000,0


Features shape (4000, 116)
Event rate: 0.139
Follow-up days: median 10.0  IQR 11.0


## 3.1 Preprocessing and leakage control
- Detect numeric vs categorical features programmatically  
- Numeric pipeline  
  - SimpleImputer with median  
- Categorical pipeline  
  - SimpleImputer with most frequent  
  - OneHotEncoder with ignore for unseen categories  
- Build one ColumnTransformer used by all models inside scikit-learn Pipelines  
- We exclude outcome variables and identifiers from the feature matrix to prevent target leakage


In [ ]:
# 3.1 Build shared preprocessor

num_cols, cat_cols = utils.detect_feature_types(X)
print("Numeric", len(num_cols), "Categorical", len(cat_cols))

preprocessor: ColumnTransformer = utils.build_preprocessor(num_cols, cat_cols)

# Fit preprocessor only on the training partition later; here we can preview on the full data safely without leaking,
# but we will re-fit strictly on train after we split
preprocessor.fit(X)
Xt_preview = utils.to_dataframe(preprocessor, X.head(200))
display(Xt_preview.head(3).style)
print("Transformed preview shape", Xt_preview.shape)


Numeric 116 Categorical 0


,num__SAPS-I,num__SOFA,num__Age,num__Gender,num__Height,num__Weight,num__CCU,num__CSRU,num__SICU,num__DiasABP_first,num__GCS_first,num__Glucose_first,num__HR_first,num__MAP_first,num__NIDiasABP_first,num__NIMAP_first,num__NISysABP_first,num__RespRate_first,num__SaO2_first,num__Temp_first,num__DiasABP_last,num__GCS_last,num__Glucose_last,num__HR_last,num__MAP_last,num__NIDiasABP_last,num__NIMAP_last,num__NISysABP_last,num__RespRate_last,num__SaO2_last,num__Temp_last,num__DiasABP_lowest,num__GCS_lowest,num__Glucose_lowest,num__HR_lowest,num__MAP_lowest,num__NIDiasABP_lowest,num__NIMAP_lowest,num__NISysABP_lowest,num__RespRate_lowest,num__SaO2_lowest,num__Temp_lowest,num__DiasABP_highest,num__GCS_highest,num__Glucose_highest,num__HR_highest,num__MAP_highest,num__NIDiasABP_highest,num__NIMAP_highest,num__NISysABP_highest,num__RespRate_highest,num__SaO2_highest,num__Temp_highest,num__DiasABP_median,num__GCS_median,num__Glucose_median,num__HR_median,num__MAP_median,num__NIDiasABP_median,num__NIMAP_median,num__NISysABP_median,num__RespRate_median,num__SaO2_median,num__Temp_median,num__ALP_first,num__ALT_first,num__AST_first,num__Albumin_first,num__BUN_first,num__Bilirubin_first,num__Cholesterol_first,num__Creatinine_first,num__FiO2_first,num__HCO3_first,num__HCT_first,num__K_first,num__Lactate_first,num__Mg_first,num__Na_first,num__PaCO2_first,num__PaO2_first,num__Platelets_first,num__SysABP_first,num__TroponinI_first,num__TroponinT_first,num__WBC_first,num__Weight_first,num__pH_first,num__ALP_last,num__ALT_last,num__AST_last,num__Albumin_last,num__BUN_last,num__Bilirubin_last,num__Cholesterol_last,num__Creatinine_last,num__FiO2_last,num__HCO3_last,num__HCT_last,num__K_last,num__Lactate_last,num__Mg_last,num__Na_last,num__PaCO2_last,num__PaO2_last,num__Platelets_last,num__SysABP_last,num__TroponinI_last,num__TroponinT_last,num__WBC_last,num__Weight_last,num__pH_last,num__MechVentStartTime,num__MechVentDuration,num__MechVentLast8Hour,num__UrineOutputSum
0,6.000000,1.000000,54.000000,0.000000,170.200000,78.700000,0.000000,0.000000,1.000000,61.000000,15.000000,205.000000,73.000000,82.000000,65.000000,92.330000,147.000000,19.000000,98.000000,35.100000,60.000000,15.000000,115.000000,86.000000,80.000000,55.000000,79.330000,128.000000,23.000000,97.000000,37.800000,44.000000,14.000000,115.000000,58.000000,58.000000,39.000000,58.670000,96.000000,12.000000,96.000000,35.100000,81.000000,15.000000,205.000000,86.000000,109.000000,67.000000,92.330000,157.000000,24.000000,98.000000,38.200000,58.000000,15.000000,160.000000,73.000000,78.000000,49.000000,70.000000,110.000000,18.000000,97.500000,37.700000,79.000000,31.000000,46.000000,3.000000,13.000000,0.700000,153.000000,0.800000,1.000000,26.000000,33.700000,4.400000,1.900000,1.500000,137.000000,40.000000,168.000000,221.000000,121.000000,2.200000,0.130000,11.200000,81.150000,7.380000,78.000000,31.000000,44.000000,3.000000,8.000000,0.700000,152.000000,0.700000,0.500000,28.000000,30.300000,4.000000,1.600000,1.900000,136.000000,39.000000,106.000000,185.000000,121.000000,2.000000,0.130000,9.400000,82.200000,7.400000,122.000000,2320.000000,1.000000,13.000000
1,16.000000,8.000000,76.000000,1.000000,175.300000,76.000000,0.000000,1.000000,0.000000,67.000000,3.000000,105.000000,88.000000,79.000000,38.000000,49.330000,72.000000,19.000000,99.000000,35.200000,51.000000,15.000000,146.000000,65.000000,69.000000,49.000000,68.330000,107.000000,19.000000,97.000000,37.100000,32.000000,3.000000,105.000000,65.000000,43.000000,38.000000,49.330000,72.000000,12.000000,93.000000,34.500000,81.000000,15.000000,146.000000,90.000000,100.000000,67.000000,88.330000,131.000000,28.000000,99.000000,37.900000,59.000000,15.000000,125.500000,80.000000,79.000000,59.000000,78.165000,115.000000,19.000000,97.000000,37.450000,79.000000,31.000000,46.000000,3.000000,16.000000,0.700000,153.000000,0.800000,1.000000,21.000000,24.700000,4.300000,1.900000,3.100000,139.000000,34.000000,344.000000,164.000000,105.000000,2.200000,0.130000,7.400000,80.

Transformed preview shape (200, 116)


## 3.2. Splitting
- Stratified split by the binary event to keep class balance stable  
- Proportions  
  - Train 60 percent  
  - Validation 20 percent  
  - Test 20 percent  
- The same indices are reused for Cox, Decision Tree, and Random Forest  
- We will always report metrics on the held-out test set and use validation only for light hyperparameter selection


In [7]:
# 3.2 Create stratified train, validation, and test splits re-used across all models

def make_splits(X: pd.DataFrame, y: pd.Series, seed: int = 42):
    # First split off test 20 percent
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=seed)
    train_val_idx, test_idx = next(sss1.split(X, y))
    X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
    y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]

    # Split train vs validation 75:25 within the remaining 80 percent to yield 60:20:20 overall
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    train_idx, val_idx = next(sss2.split(X_train_val, y_train_val))

    idx_train = X_train_val.index[train_idx]
    idx_val = X_train_val.index[val_idx]
    idx_test = X_test.index

    return idx_train, idx_val, idx_test

idx_train, idx_val, idx_test = make_splits(X, y["event_death"], seed=SEED)

print("Split sizes",
      "train", len(idx_train),
      "val", len(idx_val),
      "test", len(idx_test))

# Materialize split datasets
X_train, X_val, X_test = X.loc[idx_train], X.loc[idx_val], X.loc[idx_test]
y_train, y_val, y_test = y.loc[idx_train], y.loc[idx_val], y.loc[idx_test]

# Fit the shared preprocessor on train only
preprocessor = utils.build_preprocessor(*utils.detect_feature_types(X_train))
preprocessor.fit(X_train)

# Transformed DataFrames for convenience in Cox and for inspection
Xt_train = utils.to_dataframe(preprocessor, X_train)
Xt_val   = utils.to_dataframe(preprocessor, X_val)
Xt_test  = utils.to_dataframe(preprocessor, X_test)

print("Transformed shapes",
      Xt_train.shape, Xt_val.shape, Xt_test.shape)

# Quick leakage sanity check: confirm no outcome columns survived
assert not any(c.lower().startswith("in-hospital_death") for c in Xt_train.columns)
assert not any(c.lower().startswith("survival") for c in Xt_train.columns)
assert not any(c.lower().startswith("length_of_stay") for c in Xt_train.columns)


Split sizes train 2400 val 800 test 800
Transformed shapes (2400, 116) (800, 116) (800, 116)


# 4. Cox recap and fixed-horizon scoring

What we keep: 
- Same endpoint in-hospital death with duration_days in days and event_death as the event  
- Same preprocessing via the shared ColumnTransformer fit on train and applied to validation and test  
- Same splits

What we add now  
- Fit a single multivariable Cox model on the preprocessed training set  
- Compute fixed-horizon risks at 7, 30, and 60 days on validation and test  
- Evaluate discrimination, calibration, and overall accuracy on the identical evaluable cohorts per horizon  
- Store predictions, evaluability masks, and a tidy metrics table for later comparisons

In [16]:
# 7.1 Cox recap and fixed-horizon scoring

# Horizons in days
HORIZONS = [7, 30, 60]

# 1) Prepare modeling frames for lifelines using preprocessed matrices from earlier steps
df_train = Xt_train.copy()
df_train["duration_days"] = y_train["duration_days"].values
df_train["event_death"]   = y_train["event_death"].values

df_val = Xt_val.copy()
df_val["duration_days"] = y_val["duration_days"].values
df_val["event_death"]   = y_val["event_death"].values

df_test = Xt_test.copy()
df_test["duration_days"] = y_test["duration_days"].values
df_test["event_death"]   = y_test["event_death"].values

# 2) Fit Cox on train only
cph = CoxPHFitter()
cph.fit(df_train, duration_col="duration_days", event_col="event_death")

# 3) Cox fixed-horizon risks using utils, no renaming
cph_pred_train = utils.predict_fixed_horizon_risk_from_cox(cph, Xt_train, [7, 30, 60])
cph_pred_val   = utils.predict_fixed_horizon_risk_from_cox(cph, Xt_val,   [7, 30, 60])
cph_pred_test  = utils.predict_fixed_horizon_risk_from_cox(cph, Xt_test,  [7, 30, 60])

# 4) Fixed-horizon labels and evaluability masks using utils signature
labels_train, labels_val, labels_test = {}, {}, {}
for h in [7, 30, 60]:
    yb_tr, m_tr = utils.get_fixed_horizon_labels(y_train, horizon_days=h)
    yb_va, m_va = utils.get_fixed_horizon_labels(y_val,   horizon_days=h)
    yb_te, m_te = utils.get_fixed_horizon_labels(y_test,  horizon_days=h)
    labels_train[h] = {"y_true": pd.Series(yb_tr, index=y_train.index).astype(int),
                       "mask":   pd.Series(m_tr,  index=y_train.index).astype(bool)}
    labels_val[h]   = {"y_true": pd.Series(yb_va, index=y_val.index).astype(int),
                       "mask":   pd.Series(m_va, index=y_val.index).astype(bool)}
    labels_test[h]  = {"y_true": pd.Series(yb_te, index=y_test.index).astype(int),
                       "mask":   pd.Series(m_te, index=y_test.index).astype(bool)}

# 5) Metrics per horizon on identical evaluable cohorts
records = []
for set_name, preds, labels in [
    ("train", cph_pred_train, labels_train),
    ("val",   cph_pred_val,   labels_val),
    ("test",  cph_pred_test,  labels_test),
]:
    for h in [7, 30, 60]:
        col = f"Risk_{h}d"              # use utils naming
        m = labels[h]["mask"]
        y = labels[h]["y_true"][m].to_numpy()
        p = preds[col][m].to_numpy()
        if y.size == 0 or len(np.unique(y)) < 2:
            auroc = np.nan; auprc = np.nan; brier = np.nan
        else:
            auroc = roc_auc_score(y, p)
            auprc = average_precision_score(y, p)
            brier = brier_score_loss(y, p)
        records.append({
            "model": "Cox",
            "set": set_name,
            "horizon_days": h,
            "auroc": float(auroc) if np.isfinite(auroc) else np.nan,
            "auprc": float(auprc) if np.isfinite(auprc) else np.nan,
            "brier": float(brier) if np.isfinite(brier) else np.nan,
            "n_evaluable": int(m.sum())
        })

cph_metrics = pd.DataFrame.from_records(records).sort_values(["set", "horizon_days"])
display(cph_metrics.style)

# Optional compact calibration summaries on test using sklearn-style bins
calib_cph_test = {}
for h in [7, 30, 60]:
    m = labels_test[h]["mask"]
    y = labels_test[h]["y_true"][m].to_numpy()
    p = cph_pred_test[f"Risk_{h}d"][m].to_numpy()
    if y.size == 0 or len(np.unique(y)) < 2:
        calib_cph_test[h] = pd.DataFrame({"prob_mean": [], "event_rate": []})
    else:
        frac_pos, prob_mean = calibration_curve(y, p, n_bins=10, strategy="quantile")
        calib_cph_test[h] = pd.DataFrame({"prob_mean": prob_mean, "event_rate": frac_pos})

# Store artifacts unchanged
ARTIFACTS = {
    "cph_model": cph,
    "cph_pred_train": cph_pred_train,
    "cph_pred_val":   cph_pred_val,
    "cph_pred_test":  cph_pred_test,
    "labels_train": labels_train,
    "labels_val":   labels_val,
    "labels_test":  labels_test,
    "cph_metrics":  cph_metrics,
    "calib_cph_test": calib_cph_test,
    "horizons_days": [7, 30, 60],
}


,model,set,horizon_days,auroc,auprc,brier,n_evaluable
6,Cox,test,7,0.794999,0.249614,0.067331,596
7,Cox,test,30,0.682333,0.764004,0.291269,160
8,Cox,test,60,0.694444,0.971101,0.193629,114
0,Cox,train,7,0.856604,0.430239,0.049860,1833
1,Cox,train,30,0.784106,0.857376,0.229372,488
2,Cox,train,60,0.880734,0.989202,0.151444,353
3,Cox,val,7,0.799507,0.237835,0.054764,615
4,Cox,val,30,0.730904,0.766264,0.254967,168
5,Cox,val,60,0.747706,0.956124,0.205476,119


#### **Quick recap on metrics**
* **`auroc`** (Area Under the ROC Curve): This is a measure of **ranking**. It tells us if the model is good at giving higher-risk patients higher scores than lower-risk patients
* **`auprc`** (Area Under the Precision-Recall Curve): This is a measure of **positive prediction value**, which is very useful when the event (death) is rare. It answers: "When the model predicts a patient is high-risk, how often is it correct?"
* **`brier`**: This measures the **accuracy of the probability score itself**. It penalizes models for being overconfident or underconfident. A perfect score is 0. **Lower is better**

**Clinical takeaways**
* **It's a strong baseline**: An AUROC of ~0.80 on the test set for short-term risk is a solid result and provides genuine clinical value. It shows that the model is effective at identifying which patients are at higher risk
* **Actionability**: Before using these risk scores to make decisions (e.g., flagging a patient for a clinical review), we would also need to check its **calibration**